In [69]:
import pycparser
from pycparser import parse_file, c_generator
from pycparser.c_ast import PtrDecl

In [43]:
ast = pycparser.parse_file('header/preped')

In [160]:
funcs = ""
calls = []
for el in ast.ext:
    if el.name in getters and el.name not in failed: # "PDC_GetColorEnhanceMode":
        #print(el)
        var_defs = []
        var_params = []
        fmts = [el.name]
        var_lists = []
        child = False
        needs_param = False
        for param in el.type.args.params:
            
            if param.name  == 'nDeviceNo':
                continue
            if param.name  == 'nChildNo':
                child = True
                continue
            #print(param.name)
            if type(param.type) == PtrDecl:
                dtype = " ".join(param.type.type.type.names)
                var_defs.append(f"{dtype} {param.name};\n")
                var_params.append(f"&{param.name}")
                var_lists.append(param.name)
                fmts.append(f"{param.name},%d")
            else:
                #print(param)
                needs_param = True
        if needs_param:
            continue
        fname = el.name
        var_def = "\n".join(var_defs)
        var_param = ",".join(var_params)
        fmt = " ; ".join(fmts)
        var_list = ",".join(var_lists)
        form = f"""void test_{fname}(uint64_t interfaceId)
{{
    {var_def}
    unsigned long retVal;

    retVal = {fname}(IFACE_ID_GET_DEV_NUM(interfaceId),
                                { "IFACE_ID_GET_CHILD_NUM(interfaceId)," if child else ""}
                                {var_param});    // Output

    if (retVal == PDC_FAILED)
    {{
        printf("Failed to read {fname}: %d\\n", retVal);
        return;
    }}

    printf("{fmt}\\n",{var_list});
}}
"""
        funcs+=form
        calls.append(fname)
    elif el.name in listings and el.name not in failed:
        var_defs = []
        var_params = []
        fmts = [el.name]
        var_lists = []
        child = False
        dev = False
        needs_param = False
        for param in el.type.args.params:
            
            if param.name  == 'nDeviceNo':
                dev = True
                continue
            if param.name  == 'nChildNo':
                child = True
                continue
            #print(param.name)
            if param.name  in ['pSize','pList','pErrorCode']:
                continue

            needs_param = True
                
        if needs_param:
            continue
        fname = el.name
        var_def = "\n".join(var_defs)
        var_param = ",".join(var_params)
        fmt = " ; ".join(fmts)
        var_list = ",".join(var_lists)
        form = f"""void test_{fname}(uint64_t interfaceId)
{{
    unsigned long retVal;
    unsigned long errorCode;
    unsigned long modeCnt;
    unsigned long modeList[PDC_MAX_LIST_NUMBER];

    retVal = {fname}( { "IFACE_ID_GET_DEV_NUM(interfaceId)," if dev else ""}
                                    { "IFACE_ID_GET_CHILD_NUM(interfaceId)," if child else ""}
                                    &modeCnt,       // Output
                                    modeList,       // Output
                                    &errorCode);    // Output
    if (retVal == PDC_FAILED)
    {{
        printf("Failed to read {fname}: %d\\n", retVal);
        return;
    }}

    unsigned long i;
    printf("{el.name} LIST:");
    for (i = 0; i < modeCnt; i++)
    {{
        printf("%d ;",modeList[i]);
    }}
    printf("\\n");
}}
"""
        funcs+=form
        calls.append(fname)

In [92]:
getters

['PDC_GetDeviceCode',
 'PDC_GetDeviceName',
 'PDC_GetDeviceID',
 'PDC_GetProductID',
 'PDC_GetLotID',
 'PDC_GetIndividualID',
 'PDC_GetVersion',
 'PDC_GetEthernetInfo',
 'PDC_GetMaxChildDeviceCount',
 'PDC_GetChildDeviceCount',
 'PDC_GetCameraCommandBuadRate',
 'PDC_GetStatus',
 'PDC_GetCamMode',
 'PDC_GetMemory',
 'PDC_GetMemoryInfo',
 'PDC_GetMaxBitDepth',
 'PDC_GetBitDepth',
 'PDC_GetRecordRate',
 'PDC_GetHighSpeedMode',
 'PDC_GetMaxFullResoRate',
 'PDC_GetMaxResolution',
 'PDC_GetResolution',
 'PDC_GetResolutionLockMode',
 'PDC_GetSegmentPosition',
 'PDC_GetResolutionROI',
 'PDC_GetShutterSpeedFps',
 'PDC_GetShutterSpeedUsec',
 'PDC_GetShutterSpeedNsec',
 'PDC_GetStepShutter',
 'PDC_GetShutterLockMode',
 'PDC_GetTriggerMode',
 'PDC_GetCurrentPartition',
 'PDC_GetMaxFrames',
 'PDC_GetMaxPartition',
 'PDC_GetPartitionIncMode',
 'PDC_GetVariableChannel',
 'PDC_GetVariableChannelInfo',
 'PDC_GetVariableRestriction',
 'PDC_GetVariableMaxWidth',
 'PDC_GetVariableMaxHeight',
 'PDC_GetVari

In [91]:
getters = []
with open("header/getters.dump") as f:
    for line in f:
        if "PDC_Get" in line:
            print(line.strip())
            getters.append(line.strip())

PDC_GetDeviceCode
PDC_GetDeviceName
PDC_GetDeviceID
PDC_GetProductID
PDC_GetLotID
PDC_GetIndividualID
PDC_GetVersion
PDC_GetEthernetInfo
PDC_GetMaxChildDeviceCount
PDC_GetChildDeviceCount
PDC_GetCameraCommandBuadRate
PDC_GetStatus
PDC_GetCamMode
PDC_GetMemory
PDC_GetMemoryInfo
PDC_GetMaxBitDepth
PDC_GetBitDepth
PDC_GetRecordRate
PDC_GetHighSpeedMode
PDC_GetMaxFullResoRate
PDC_GetMaxResolution
PDC_GetResolution
PDC_GetResolutionLockMode
PDC_GetSegmentPosition
PDC_GetResolutionROI
PDC_GetShutterSpeedFps
PDC_GetShutterSpeedUsec
PDC_GetShutterSpeedNsec
PDC_GetStepShutter
PDC_GetShutterLockMode
PDC_GetTriggerMode
PDC_GetCurrentPartition
PDC_GetMaxFrames
PDC_GetMaxPartition
PDC_GetPartitionIncMode
PDC_GetVariableChannel
PDC_GetVariableChannelInfo
PDC_GetVariableRestriction
PDC_GetVariableMaxWidth
PDC_GetVariableMaxHeight
PDC_GetVariableMaxRate
PDC_GetVariableMaxResolution
PDC_GetAutoExposure
PDC_GetAutoExposureInfo
PDC_GetAutoExposureRestriction
PDC_GetLowLightMode
PDC_GetColorType
PDC_GetCo

In [147]:
listings = []
with open("header/lists.dump") as f:
    for line in f:
        if "PDC_" in line:
            print(line.strip())
            listings.append(line.strip())

PDC_GetExistChildDeviceList
PDC_GetOpenedDeviceList
PDC_GetRecordRateList
PDC_GetResolutionList
PDC_GetShutterSpeedFpsList
PDC_GetShutterSpeedUsecList
PDC_GetShutterSpeedNsecList
PDC_GetTriggerModeList
PDC_GetColorTempModeList
PDC_GetVideoOutModeList
PDC_GetVideoOutSignalList
PDC_GetVideoOutHDSDIList
PDC_GetExternalInModeList
PDC_GetExternalOutModeList
PDC_GetSensorGainModeList
PDC_GetSensorGammaModeList
PDC_GetPartitionList
PDC_GetDSShutterModeList
PDC_GetLUTModeList
PDC_GetLUTOrgValueList
PDC_GetLUTValueList
PDC_GetShadingModeList
PDC_GetShadingTypeList
PDC_GetPixelGainModeList
PDC_GetEdgeEnhanceModeList
PDC_GetColorEnhanceModeList
PDC_GetDownloadModeList
PDC_GetVariableRecordRateList
PDC_GetAEShutterFpsList
PDC_GetAEShutterUsecList
PDC_GetAEShutterNsecList
PDC_GetProgSwitchModeList
PDC_GetSyncOutTimesList
PDC_GetCameraCommandBaudRateList
PDC_GetHeadExchangeList
PDC_GetSyncPriorityList
PDC_GetDropFrameList
PDC_GetBlackClipLevelList
PDC_GetRecordingTypeList
PDC_GetPartitionIncModeList

In [161]:
with open("funcs.c", "w") as f:
    f.write(f"""
    #include <windows.h>
#include <stdio.h>
#include <inttypes.h>
#include <stdlib.h>
#include <limits.h>

#include <string>
#include <sstream>

#include "PDCLIB.h"


#define IFACE_ID_FIELD_OFFSET 32
#define IFACE_ID_FROM_VALUES(deviceNum, childNum) ((((uint64_t)deviceNum) << IFACE_ID_FIELD_OFFSET) + childNum)
#define IFACE_ID_GET_DEV_NUM(interfaceId) ((unsigned long)(interfaceId >> IFACE_ID_FIELD_OFFSET))
#define IFACE_ID_GET_CHILD_NUM(interfaceId) ((unsigned long)(interfaceId & (((uint64_t)1 << IFACE_ID_FIELD_OFFSET) - 1)))
""")
    f.write(funcs)

    f.write(f"""
    void test_all(uint64_t dev) 
{{
{callstr}
}}
    
    """)

In [112]:
print(funcs)

void test_PDC_GetDeviceCode(uint64_t interfaceId)
{
    unsigned long pCode;

unsigned long pErrorCode;

    unsigned long retVal;

    retVal = PDC_GetDeviceCode(IFACE_ID_GET_DEV_NUM(interfaceId),
                                
                                &pCode,&pErrorCode);    // Output

    if (retVal == PDC_FAILED)
    {
        printf("Failed to read PDC_GetDeviceCode: %d!", retVal);
    }

    printf("PDC_GetDeviceCode ; pCode,%d ; pErrorCode,%d\n",pCode,pErrorCode);
}
void test_PDC_GetDeviceID(uint64_t interfaceId)
{
    unsigned long pID;

unsigned long pErrorCode;

    unsigned long retVal;

    retVal = PDC_GetDeviceID(IFACE_ID_GET_DEV_NUM(interfaceId),
                                
                                &pID,&pErrorCode);    // Output

    if (retVal == PDC_FAILED)
    {
        printf("Failed to read PDC_GetDeviceID: %d!", retVal);
    }

    printf("PDC_GetDeviceID ; pID,%d ; pErrorCode,%d\n",pID,pErrorCode);
}
void test_PDC_GetProductID(uint64_t interf

In [155]:
callstr = "\n".join(map(lambda x:f"test_{x}(dev);",calls))

In [156]:
print(callstr)

test_PDC_GetRecordRateList(dev);
test_PDC_GetResolutionList(dev);
test_PDC_GetShutterSpeedFpsList(dev);
test_PDC_GetShutterSpeedUsecList(dev);
test_PDC_GetShutterSpeedNsecList(dev);
test_PDC_GetTriggerModeList(dev);
test_PDC_GetSensorGainModeList(dev);
test_PDC_GetSensorGammaModeList(dev);
test_PDC_GetColorTempModeList(dev);
test_PDC_GetLUTModeList(dev);
test_PDC_GetVariableRecordRateList(dev);
test_PDC_GetDSShutterModeList(dev);
test_PDC_GetVideoOutModeList(dev);
test_PDC_GetShadingModeList(dev);
test_PDC_GetAEShutterFpsList(dev);
test_PDC_GetAEShutterUsecList(dev);
test_PDC_GetAEShutterNsecList(dev);
test_PDC_GetEdgeEnhanceModeList(dev);
test_PDC_GetSyncPriorityList(dev);
test_PDC_GetExistChildDeviceList(dev);
test_PDC_GetOpenedDeviceList(dev);
test_PDC_GetProgSwitchModeList(dev);
test_PDC_GetPixelGainModeList(dev);
test_PDC_GetSyncOutTimesList(dev);
test_PDC_GetColorEnhanceModeList(dev);
test_PDC_GetHeadExchangeList(dev);
test_PDC_GetCameraCommandBaudRateList(dev);
test_PDC_GetDownl

In [125]:
output = b'Hello World! from C\r\ninit error code 1, ret val 1check auto\r\nip in numberc 0xc0a87dc8\r\nfound 1 devices device code 39\r\nburst supported\r\nsleep 1s\r\nslept\r\nPDC_GetDeviceCode ; pCode,39 ; pErrorCode,1\r\nPDC_GetDeviceID ; pID,200 ; pErrorCode,1\r\nFailed to read PDC_GetProductID: 0\r\nFailed to read PDC_GetLotID: 0\r\nFailed to read PDC_GetIndividualID: 0\r\nPDC_GetVersion ; pVer,459271 ; pErrorCode,1\r\nPDC_GetColorType ; pMode,0 ; pErrorCode,1\r\nFailed to read PDC_GetStatus: 0\r\nPDC_GetShutterLockMode ; pMode,0 ; pErrorCode,1\r\nPDC_GetResolutionLockMode ; pMode,0 ; pErrorCode,1\r\nPDC_GetPartitionIncMode ; pMode,0 ; pErrorCode,1\r\nFailed to read PDC_GetKeyPadLock: 0\r\nFailed to read PDC_GetVideoOutMode: 0\r\nPDC_GetChildDeviceCount ; pCount,1 ; pErrorCode,1\r\nPDC_GetMaxChildDeviceCount ; pCount,1 ; pErrorCode,1\r\nPDC_GetExternalCount ; pIn,4 ; pOut,2 ; pErrorCode,1\r\nPDC_GetIRIG ; pMode,0 ; pErrorCode,1\r\nFailed to read PDC_GetMCDL: 0\r\nPDC_GetIRIGOffset ; pValue,0 ; pErrorCode,1\r\nPDC_GetRecordRate ; pRate,16000 ; pErrorCode,1\r\nPDC_GetTriggerMode ; pMode,0 ; pAFrames,21821 ; pRFrames,0 ; pRCount,0 ; pErrorCode,1\r\nPDC_GetResolution ; pWidth,1024 ; pHeight,1024 ; pErrorCode,1\r\nFailed to read PDC_GetResolutionROI: 0\r\nPDC_GetShutterSpeedFps ; pFps,16000 ; pErrorCode,1\r\nPDC_GetShutterSpeedUsec ; pUsec,60 ; pErrorCode,1\r\nPDC_GetShutterSpeedNsec ; pNsec,60700 ; pErrorCode,1\r\nPDC_GetMaxPartition ; pCount,128 ; pBlock,0 ; pErrorCode,1\r\nPDC_GetCurrentPartition ; pNo,1 ; pErrorCode,1\r\nFailed to read PDC_GetSensorGainMode: 0\r\nFailed to read PDC_GetSensorGammaMode: 0\r\nFailed to read PDC_GetColorTempMode: 0\r\nPDC_GetAutoExposure ; pMode,0 ; pErrorCode,1\r\nPDC_GetLUTMode ; pMode,1 ; pErrorCode,1\r\nPDC_GetDSShutterMode ; pMode,17 ; pErrorCode,1\r\nPDC_GetDSShutterValue ; pValue,0 ; pErrorCode,1\r\nPDC_GetEdgeEnhanceMode ; pMode,1 ; pErrorCode,1\r\nFailed to read PDC_GetShadingMode: 0\r\nPDC_GetPixelGainMode ; pMode,1 ; pErrorCode,1\r\nPDC_GetMaxFrames ; pFrames,21821 ; pBlocks,0 ; pErrorCode,1\r\nPDC_GetMaxResolution ; pWidth,1024 ; pHeight,1024 ; pErrorCode,1\r\nPDC_GetSegmentPosition ; pXPos,0 ; pYPos,0 ; pErrorCode,1\r\nPDC_GetVariableChannel ; pChannel,0 ; pErrorCode,1\r\nPDC_GetAutoExposureInfo ; pWidth,128 ; pHeight,16 ; pXPos,0 ; pYPos,0 ; pValue,0 ; pRange,0 ; pLimit,0 ; pErrorCode,1\r\nPDC_GetMaxBitDepth ; pDepth,12 ; pErrorCode,1\r\nPDC_GetVariableRestriction ; pWidthStep,128 ; pHeightStep,16 ; pXPosStep,64 ; pYPosStep,8 ; pWidthMin,128 ; pHeightMin,16 ; pFreePos,0 ; pErrorCode,1\r\nFailed to read PDC_GetColorEnhanceMode: 0\r\nPDC_GetDownloadMode ; pMode,2 ; pErrorCode,1\r\nPDC_GetCamMode ; pMode,0 ; pErrorCode,1\r\nPDC_GetMaxFullResoRate ; pRate,16000 ; pErrorCode,1\r\nPDC_GetLowLightMode ; pMode,0 ; pErrorCode,1\r\nFailed to read PDC_GetColorTempBase: 0\r\nPDC_GetAutoExposureRestriction ; pWidthStep,64 ; pHeightStep,16 ; pXPosStep,32 ; pYPosStep,8 ; pWidthMin,128 ; pHeightMin,16 ; pMaxValue,1023 ; pMaxRange,1023 ; pErrorCode,1\r\nPDC_GetSyncOutTimes ; pRatio,1 ; pErrorCode,1\r\nPDC_GetStoreNum ; pNum,1 ; pErrorCode,1\r\nPDC_GetIRIGPhaseLock ; pMode,0 ; pErrorCode,1\r\nPDC_GetIRIGPhaseLockStatus ; pMode,0 ; pErrorCode,1\r\nFailed to read PDC_GetHighSpeedMode: 0\r\nPDC_GetBurstTransfer ; pMode,1 ; pErrorCode,1\r\nFailed to read PDC_GetHeadExchange: 0\r\nFailed to read PDC_GetStepShutter: 0\r\nPDC_GetBitDepth ; pDepth,12 ; pErrorCode,1\r\nFailed to read PDC_GetSyncPriority: 0\r\nFailed to read PDC_GetVideoOutSignal: 0\r\nFailed to read PDC_GetVideoOutSignalEnable: 0\r\nPDC_GetVideoOutHDSDI ; pMode,3 ; pErrorCode,1\r\nPDC_GetRecordingType ; pMode,2 ; pErrorCode,1\r\nFailed to read PDC_GetAutoPlay: 0\r\nPDC_GetFactoryDefaults ; pMode,0 ; pErrorCode,1\r\nPDC_GetStorePreset ; pNo,1 ; pErrorCode,1\r\nPDC_GetStorePresetReboot ; pReboot,0 ; pErrorCode,1\r\nPDC_GetInstructionSet ; pInstSetCode,-2147483648 ; pErrorCode,1\r\nFailed to read PDC_GetLEDMode: 0\r\nFailed to read PDC_GetPolarizerConfig: 0\r\nFailed to read PDC_GetShadingType: 0\r\nFailed to read PDC_GetBlackClipLevel: 0\r\nPDC_GetMemoryModePartition ; pNo,1 ; pErrorCode,1\r\nPDC_GetLensControl ; pType,3 ; pErrorCode,1\r\nFailed to read PDC_GetFanState: 0\r\nPDC_GetTransferOption ; p8BitSel,4 ; pBayer,0 ; pInterleave,1 ; pErrorCode,1\r\nFailed to read PDC_GetLensInfo: 0\r\nPDC_GetSelfSyncMode ; pMode,0 ; pErrorCode,1\r\nPDC_GetMaxRandomTriggerCount ; pMaxRandomCount,10 ; pErrorCode,1\r\nPDC_GetMaxRandomEndlessTriggerCount ; pMaxRandomCount,10 ; pErrorCode,1\r\nFailed to read PDC_GetAccelerationSensorError: 0\r\nFailed to read PDC_GetBayerAlignment: 0\r\nFailed to read PDC_GetMemory: 0\r\nPDC_GetReadAheadTransfer ; pMode,0 ; pErrorCode,1\r\nPDC_GetLowLightMode2 ; pMode,0 ; pErrorCode,1\r\nFailed to read PDC_GetWiFiScanCount: 0\r\nFailed to read PDC_GetShutdownBatterySupply: 0\r\nFailed to read PDC_GetShutdownBatterySupplyRange: 0\r\nFailed to read PDC_GetAutoPowerWhenSupply: 0\r\nFailed to read PDC_GetAccelerationTrigger: 0\r\nFailed to read PDC_GetAccelerationTriggerTime: 0\r\nFailed to read PDC_GetAccelerationTriggerTimeLimit: 0\r\nPDC_GetFanRecReadyStop ; pFlag,0 ; pErrorCode,1\r\nFailed to read PDC_GetTriggerNoiseFilter: 0\r\nFailed to read PDC_GetTriggerNoiseFilterWidth: 0\r\nFailed to read PDC_GetMemoryShare: 0\r\nFailed to read PDC_GetVideoOSDCrosshairMode: 0\r\nFailed to read PDC_GetSerialNumber: 0\r\nPDC_GetShutterSpeedPrecision ; pValue,1 ; pErrorCode,1\r\nFailed to read PDC_GetAutoExposureLimit: 0\r\nFailed to read PDC_GetCameraHeadType: 0\r\nFailed to read PDC_GetHeadPortNumber: 0\r\nPDC_GetLinkSpeed ; pLinkSpeed,0 ; pErrorCode,1\r\nPDC_GetEthernetSpeedLimit ; pParam,1 ; pErrorCode,1\r\ncapture rate 16000'

In [133]:
failed = []
for line in output.decode().split("\n"):
    if line.startswith("Failed to read"):
        fname = line[15:].split(":")[0]
        failed.append(fname)

In [134]:
failed

['PDC_GetProductID',
 'PDC_GetLotID',
 'PDC_GetIndividualID',
 'PDC_GetStatus',
 'PDC_GetKeyPadLock',
 'PDC_GetVideoOutMode',
 'PDC_GetMCDL',
 'PDC_GetResolutionROI',
 'PDC_GetSensorGainMode',
 'PDC_GetSensorGammaMode',
 'PDC_GetColorTempMode',
 'PDC_GetShadingMode',
 'PDC_GetColorEnhanceMode',
 'PDC_GetColorTempBase',
 'PDC_GetHighSpeedMode',
 'PDC_GetHeadExchange',
 'PDC_GetStepShutter',
 'PDC_GetSyncPriority',
 'PDC_GetVideoOutSignal',
 'PDC_GetVideoOutSignalEnable',
 'PDC_GetAutoPlay',
 'PDC_GetLEDMode',
 'PDC_GetPolarizerConfig',
 'PDC_GetShadingType',
 'PDC_GetBlackClipLevel',
 'PDC_GetFanState',
 'PDC_GetLensInfo',
 'PDC_GetAccelerationSensorError',
 'PDC_GetBayerAlignment',
 'PDC_GetMemory',
 'PDC_GetWiFiScanCount',
 'PDC_GetShutdownBatterySupply',
 'PDC_GetShutdownBatterySupplyRange',
 'PDC_GetAutoPowerWhenSupply',
 'PDC_GetAccelerationTrigger',
 'PDC_GetAccelerationTriggerTime',
 'PDC_GetAccelerationTriggerTimeLimit',
 'PDC_GetTriggerNoiseFilter',
 'PDC_GetTriggerNoiseFilterW

In [146]:
for line in output.decode().split("\n"):
    if line.startswith("Failed to read"):
        continue
    if line.startswith("PDC_"):
        print(line.split(";")[0:-1])

['PDC_GetDeviceCode ', ' pCode,39 ']
['PDC_GetDeviceID ', ' pID,200 ']
['PDC_GetVersion ', ' pVer,459271 ']
['PDC_GetColorType ', ' pMode,0 ']
['PDC_GetShutterLockMode ', ' pMode,0 ']
['PDC_GetResolutionLockMode ', ' pMode,0 ']
['PDC_GetPartitionIncMode ', ' pMode,0 ']
['PDC_GetChildDeviceCount ', ' pCount,1 ']
['PDC_GetMaxChildDeviceCount ', ' pCount,1 ']
['PDC_GetExternalCount ', ' pIn,4 ', ' pOut,2 ']
['PDC_GetIRIG ', ' pMode,0 ']
['PDC_GetIRIGOffset ', ' pValue,0 ']
['PDC_GetRecordRate ', ' pRate,16000 ']
['PDC_GetTriggerMode ', ' pMode,0 ', ' pAFrames,21821 ', ' pRFrames,0 ', ' pRCount,0 ']
['PDC_GetResolution ', ' pWidth,1024 ', ' pHeight,1024 ']
['PDC_GetShutterSpeedFps ', ' pFps,16000 ']
['PDC_GetShutterSpeedUsec ', ' pUsec,60 ']
['PDC_GetShutterSpeedNsec ', ' pNsec,60700 ']
['PDC_GetMaxPartition ', ' pCount,128 ', ' pBlock,0 ']
['PDC_GetCurrentPartition ', ' pNo,1 ']
['PDC_GetAutoExposure ', ' pMode,0 ']
['PDC_GetLUTMode ', ' pMode,1 ']
['PDC_GetDSShutterMode ', ' pMode,17 ']
